# Step1: Scrape Tweets

### Use Tweepy API to search tweets. Only return a few days' tweets due to Twitter API limitations

In [ ]:
import tweepy

In [ ]:
# Specify the account credentials in the following variables:
consumer_key = '6LH2LIMPGNWMxNE2a6LfILp7i'
consumer_secret = '0eWTMNdazgVNCgt1lRCQapeN8O54GeonyGHREyebYQs7dL3upn'
access_token = '78610351-PaQIZu59zNjgIX7OeaKlU1Jr9LMqCTxVmGa7vxLZM'
access_token_secret = 'J32gDj3Vdi3bjB0IbisO0IoCKWadDogVeGgZSyv510jZ4'


# Authenticate
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True) # 'wait on rate limit' is to avoid request failure

In [ ]:
con=0
for tweet in tweepy.Cursor(api.search, q={"#currys"}, count=10,
                           lang="en", since="2018-03-10").items():
    con+=1
    print("\n-----\n"+str(con)+"\n"+str(tweet.created_at), tweet.text+ "\n-----\n")
    with open('currys.txt','a',encoding="utf-8") as f:
        f.write(tweet.text)
        f.write('\n')

### Use GetOldTweets3, a tool developed by some random guy in github, we can get old tweets. But much slower.

In [ ]:
import GetOldTweets3 as got

In [ ]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('#currys')\
                                           .setSince("2018-03-10")\
                                           .setUntil("2019-03-10")\
                                           .setMaxTweets(10000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
print(tweet.text)

In [ ]:
for i in range(10000):
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)[i]
    print(i)
    with open('currys.txt','a',encoding="utf-8") as f:
        f.write(tweet.text)
        f.write('\n')

### Use Selenium to get tweets. Still slow because we have to wait for loading.

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
import re

for Currys, I combine @currys and #currys results (1 year).  
1) for @currys, only 180 tweets  
2) for #currys, 2715 tweets  
3) combine them and delete duplicates, we got 2890 tweets

In [ ]:
# for headless mode
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--disable-gpu')
#browser = webdriver.Chrome(chrome_options=chrome_options)

# for no image loading
chromeOptions = webdriver.ChromeOptions()
prefs = {'profile.managed_default_content_settings.images':2}
chromeOptions.add_experimental_option("prefs", prefs)
browser = webdriver.Chrome(chrome_options=chromeOptions)
# launch the browser
#browser = webdriver.Chrome()
# currys url
#url = u'https://twitter.com/search?f=tweets&vertical=default&q=%23currys&src=typd'
# ao url
url = u'https://twitter.com/search?f=tweets&vertical=default&q=%40ao&src=typd&lang=zh-cn'
browser.get(url)


# Let the Tweets load
time.sleep(1)

# Find the body of the HTML page
body = browser.find_element_by_tag_name('body')

# Keep scrolling down using a simulation of the PAGE_DOWN button
for _ in range(1500):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)



In [ ]:
# Get the tweets by their class (similar to Beautifulsoup's find())
#tweets = browser.find_elements_by_class_name('tweet-text')
# Print Tweets
count=0
for tweet in tweets:
    count+=1
    print(count)
    
    review=re.sub('\n',' ',tweet.text) # make each tweet a single paragraph so we can count later
    #print(review)
    with open('ao.txt','a',encoding="utf-8") as f:
        f.write(review)
        f.write('\n')

In [ ]:
browser.quit()

In [ ]:
# first open #currys and count the number
with open('currys.txt','r',encoding='utf-8') as f:
    currys=f.readlines()
count=0
for i in currys:
    count+=1
count

In [ ]:
# then open @currys 
#add comments got from @currys and delete the duplicates
with open('currys_1.txt','r',encoding='utf-8') as f:
    currys_1=f.readlines()
# create a file to contain the filtered data
count=0
small_currys=[]
for i in currys_1:
    if '#currys' not in i:
        count+=1
        small_currys.append(i)
count

In [ ]:
# combine the two
currys_com=[]
currys_com=currys+small_currys
# count the number and read into new file
count=0
for i in currys_com:
    count+=1
    with open('currys_com.txt','a',encoding='utf-8') as f:
        f.write(i)
        f.write('\n')
count

## To deal with crashing, we can do it mannually.

Take facebook data as an example. We can mannually scroll down and load the page before scraping.

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

browser = webdriver.Chrome()
# change the URL to the one you want to open
url = u'https://www.facebook.com/pg/AOLetsGo/community/?ref=page_internal'

browser.get(url)
time.sleep(1)
body = browser.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)
time.sleep(1)    
body.send_keys(Keys.END)
time.sleep(2)
browser.find_element_by_link_text("Not Now").click();

In [ ]:
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')
count=0
rating = soup.find_all('p')
timestamp = soup.find_all('span',attrs = {'class' : 'timestampContent'})

for ite in rating:
    if(ite.text == "..."):
        print(" ")
    else: 
        print(str(count))
        print(ite.text)
        count=count+1
        with open('facebook_ao.txt','a',encoding='utf-8') as f:
            f.write(ite.text)
            f.write('\n-----\n')

In [ ]:
browser.quit()

# Step 2: Frequency analysis, wordcloud

### Purpose: find the words customers mentioned most frequently, so we can gain some insight on what customers care about

In [ ]:
import nltk
import re
from wordcloud import WordCloud

In [ ]:
# treat the text as bag of words
with open('currys.txt','r',encoding='utf-8') as f:
    currys=f.read()

Clean the data -- url, non-English character

In [ ]:
# clean url
currys_clean = re.sub("(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]", "", currys)             

In [ ]:
# clean non-English characters
currys_clean1 = re.sub('[^A-Za-z]',' ',currys_clean)

##### Create stop words based on frequency

In [ ]:
# create stopwords list for this case, keep updating according to output
stopword_freq=["I","i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
           "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", 
           "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", 
           "their", "theirs", "themselves", "what", "which", "who", "whom", "this",
           "that", "these", "those", "am", "is", "are", "was", "were", "be", "been",
           "being", "have", "has", "had", "having", "do", "does", "did", "doing", 
           "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
           "of", "at", "by", "for", "with", "about", "against", "between", "into",
           "through", "during", "before", "after", "above", "below", "to", "from",
           "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", 
           "then", "once", "here", "there", "when", "where", "why", "how", "all", "any",
           "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", 
           "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
           "will", "just", "don", "should", "now","via","smart","get","black","entertainment","ireland","london","england",
              "scotland","wales","londonislovinit","one","gb","marked","white","new","today","bundle","still","day","back",
              "buy","selected","logik","days","told","uk","home","never","ve","core","even","would","half","got","gold",
              "win","warriors","take","red","go","month","know","lg","jvc","ever","bought","friday","extra","shop",
              "tb","built","week","range","data","re","people","last","rt","need","stephen","night","want"]

In [ ]:
# tokenize
tokens = nltk.word_tokenize(currys_clean1)
# change all tokens into lower case
tokens = [w.lower() for w in tokens]

In [ ]:
# remove stop words
filtered_words = [word for word in tokens if word not in stopword_freq]
count=0
for i in filtered_words:
    count+=1
count

In [ ]:
# display the most frequent 25 words that has more than 1 letter
freq_words=nltk.FreqDist(w for w in filtered_words if len(w)>1)
print(freq_words.most_common(100))

### Create word cloud

In [ ]:
# delete currys for word cloud cuz they dont contain information
stopword_cloud=stopword_freq+["currys","pc","world","curryspcworld"]

In [ ]:
# use this liine if we use mask
from scipy.misc import imread

In [ ]:
# use this line if we use customed color
import numpy as np

In [ ]:
#bg_pic = imread('uk.png')
wc = WordCloud( width=2000, height=1200, margin=2,background_color="white", max_words=100, 
               stopwords=stopword_cloud)
wc.generate(currys_clean1)




In [ ]:
# design color function
def grey_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl(230,100%%, %d%%)" % np.random.randint(1,40))


Some interesting words and patterns show up.  
1) It seems that a large proportion of tweets are promotions via LatestDeal  
2) TV seems to be star product

In [ ]:
#wc.recolor(color_func = grey_color_func)
import matplotlib.pyplot as plt
plt.figure( figsize=(20,10) )
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

#### Have a look at the number of tweets excluding lduk promotion

In [ ]:
with open('currys.txt','r',encoding='utf-8') as f:
    currys=f.readlines()
# clean each review
currys_clean=[]
for review in currys:
    reivew=re.sub("(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]", "", review)
    review=re.sub('[^A-Za-z]',' ',review)
    currys_clean.append(review)
currys_clean=[review for review in currys_clean if review.strip()!='']

Count all tweets (after cleaning and excluding empty string) -- 4814

In [ ]:
count_all=0
for i in currys_clean:
    count_all+=1
count_all

Count tweets without promotion tweets --3502

In [ ]:
count_cus=0
for review in currys_clean:
    if 'lduk' not in review:
        count_cus+=1
count_cus

Therefore, only 73% of tweets are from customers

# Step 3: Sentiment Analysis

In [452]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [453]:
import re

In [454]:
currys=[]
with open('currys.txt','r',encoding='utf-8') as f:
    currys=f.readlines()

count=0
for i in currys:
    count+=1
count

2890

In [ ]:
# get half a year data from currys
currys6=[]
for i in range(1445):
    currys6.append(currys[i])
count=0
for i in currys6:
    count+=1
count

In [ ]:
ao=[]
with open('ao.txt','r',encoding='utf-8') as f:
    ao=f.readlines()
count=0
for i in ao:
    count+=1
count

In [ ]:
argos=[]
with open('argos.txt','r',encoding='utf-8') as f:
    argos=f.readlines()
count=0
for i in argos:
    count+=1
count

cleaning the data

In [455]:
# clean each review
currys_clean=[]
for review in currys:
    reivew=re.sub("(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]", "", review)
    review=re.sub('[^A-Za-z]',' ',review)
    currys_clean.append(review)
currys_clean=[review for review in currys_clean if review.strip()!='']
count=0
for i in currys_clean:
    count+=1
count

2888

In [ ]:
ao_clean=[]
for review in ao:
    reivew=re.sub("(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]", "", review)
    review=re.sub('[^A-Za-z]',' ',review)
    ao_clean.append(review)
ao_clean=[review for review in ao_clean if review.strip()!='']
count=0
for i in ao_clean:
    count+=1
count

In [ ]:
argos_clean=[]
for review in argos:
    reivew=re.sub("(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]", "", review)
    review=re.sub('[^A-Za-z]',' ',review)
    argos_clean.append(review)
argos_clean=[review for review in argos_clean if review.strip()!='']
count=0
for i in argos_clean:
    count+=1
count

In [ ]:
# delete non-customer tweets recursively according to results for currys
tokens_currys=[]
for review in currys_clean:
    token=nltk.word_tokenize(review)
    filtered_words = [word for word in token if word not in stopword and len(word)>2]
    tokens_currys=tokens_currys+filtered_words

In [ ]:
freq_words_currys=nltk.FreqDist(tokens_currys)

In [ ]:
common_words_currys=freq_words_currys.most_common(30)
print(common_words_currys)

In [456]:
# remove promotion, only keep customer reviews
currys_clean=[review for review in currys_clean if 'lduk' not in review]
currys_clean=[review for review in currys_clean if 'PromoCode' not in review]
currys_clean=[review for review in currys_clean if 'food' not in review]
currys_clean=[review for review in currys_clean if 'Food' not in review]
currys_clean=[review for review in currys_clean if 'Smart Tech' not in review]
currys_clean=[review for review in currys_clean if 'Stephen Curry' not in review]
count=0
for i in currys_clean:
    count+=1
count

1268

In [ ]:
# delete non-customer tweets recursively according to results for ao
tokens_ao=[]
for review in ao_clean:
    token=nltk.word_tokenize(review)
    filtered_words = [word for word in token if word not in stopword and len(word)>2]
    tokens_ao=tokens_ao+filtered_words

In [ ]:
freq_words_ao=nltk.FreqDist(tokens_ao)
common_words_ao=freq_words_ao.most_common(30)
print(common_words_ao)

In [ ]:
# remove
ao_clean=[review for review in ao_clean if 'AOWin' not in review]
ao_clean=[review for review in ao_clean if 'DM' not in review]
ao_clean=[review for review in ao_clean if 'Rayhaan' not in review]
ao_clean=[review for review in ao_clean if 'Rob' not in review]
ao_clean=[review for review in ao_clean if 'FreebieFriday' not in review]
ao_clean=[review for review in ao_clean if 'customersvoice' not in review]
count=0
for i in ao_clean:
    count+=1
count

In [ ]:
# delete non-customer tweets recursively according to results for argos
tokens_argos=[]
for review in argos_clean:
    token=nltk.word_tokenize(review)
    filtered_words = [word for word in token if word not in stopword and len(word)>2]
    tokens_argos=tokens_argos+filtered_words

In [ ]:
freq_words_argos=nltk.FreqDist(tokens_argos)
common_words_argos=freq_words_argos.most_common(30)
print(common_words_argos)

In [ ]:
# remove
argos_clean=[review for review in argos_clean if 'CFL' not in review]
argos_clean=[review for review in argos_clean if 'Giftcards' not in review]
argos_clean=[review for review in argos_clean if 'que' not in review]
argos_clean=[review for review in argos_clean if 'hiring' not in review]
count=0
for i in argos_clean:
    count+=1
count

In [ ]:
for i in argos_clean:
    if 'utm' in i:
        print(i)
        print('----------')

after cleaning the data, calculate sentiment scores

In [457]:
# Create a sentiment intensity analyzer object:
sid = SIA()
total_count=0
neg_count=0
pos_count=0
neu_count=0

neg_review=[]
pos_review=[]
neu_review=[]

# Loop the sentences
for review in currys_clean:
    ss = sid.polarity_scores(review) 
    #print(ss)
    total_count+=1
    if ss['compound'] < 0:
        neg_count+=1
        neg_review.append(review)
        #print(sentence, ' is overall negative ', ss['compound'])
    elif ss['compound'] == 0:
        neu_count+=1
        neu_review.append(review)
        #print(sentence, ' is overall neutral')
    else:
        pos_count+=1
        pos_review.append(review)
        #print(sentence, ' is overall positive ', ss['compound'])

Calculate the proportion of different reviews

In [458]:
print('total_count',total_count,'\n','neg_count',neg_count,'\n','pos_count',pos_count,'\n','neu_count',neu_count)

total_count 1268 
 neg_count 459 
 pos_count 541 
 neu_count 268


### Analyze the frequency of positive and negative tweets

In [459]:
# Change everything to lowercase
neg_review = [review.lower() for review in neg_review]
pos_review = [review.lower() for review in pos_review]

In [ ]:
# create a stopword list for this
stopword=["I","i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
           "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", 
           "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", 
           "their", "theirs", "themselves", "what", "which", "who", "whom", "this",
           "that", "these", "those", "am", "is", "are", "was", "were", "be", "been",
           "being", "have", "has", "had", "having", "do", "does", "did", "doing", 
           "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
           "of", "at", "by", "for", "with", "about", "against", "between", "into",
           "through", "during", "before", "after", "above", "below", "to", "from",
           "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", 
           "then", "once", "here", "there", "when", "where", "why", "how", "all", "any",
           "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", 
           "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
           "will", "just", "don", "should", "now","via","smart","get","black","entertainment","ireland","london","england",
              "scotland","wales","londonislovinit","one","gb","marked","white","new","today","bundle","still","day","back",
              "selected","logik","told","uk","home","ve","core","even","would","half","got","gold",
              "win","warriors","take","red","go","month","know","lg","jvc","ever","bought","extra",
              "tb","built","range","data","re","last","rt","need","stephen","night","want",
         "www","http","https"]
    

In [ ]:
# create tokens and bigrams for review
# for negative reviews
tokens_neg=[]
bigrams_neg=[]
for review in neg_review:
    token=nltk.word_tokenize(review)
    filtered_words = [word for word in token if word not in stopword_topic and len(word)>2]
    tokens_neg=tokens_neg+filtered_words

bigrams_neg=list(nltk.bigrams(tokens_neg))
  
# for positive reviews
tokens_pos=[]
bigrams_pos=[]
for review in pos_review:
    token=nltk.word_tokenize(review)
    filtered_words = [word for word in token if word not in stopword_topic and len(word)>2]
    tokens_pos=tokens_pos+filtered_words

bigrams_pos=list(nltk.bigrams(tokens_pos))

In [ ]:
# get frequent words and bigrams
freq_words_neg=nltk.FreqDist(tokens_neg)
freq_bigrams_neg=nltk.FreqDist(bigrams_neg)
freq_words_pos=nltk.FreqDist(tokens_pos)
freq_bigrams_pos=nltk.FreqDist(bigrams_pos)

In [ ]:
# most common words for negative reviews
common_words_neg=freq_words_neg.most_common(20)
print(common_words_neg)

In [ ]:
# most common bigrams for negative reviews
common_bigrams_neg=freq_bigrams_neg.most_common(20)
print(common_bigrams_neg)

In [ ]:
# most common words for positive reviews
common_words_pos=freq_words_pos.most_common(20)
print(common_words_pos)

In [ ]:
# most common bigrams for positive reviews
common_bigrams_pos=freq_bigrams_pos.most_common(20)
print(common_bigrams_pos)

In [ ]:
for i in currys_clean:
    if '' in i:
        print(i)

### Create wordcloud for negative reviews

In [ ]:
from wordcloud import WordCloud

In [ ]:
neg_tweets=''
for review in neg_review:
    neg_tweets=neg_tweets+' '+review
neg_tweets

In [ ]:
stopword_cloud=stopword+['currys','curryspcworld','pc','world','teamknowhowuk','pcworld','teamknowhow']

In [ ]:
#bg_pic = imread('uk.png')
wc = WordCloud( width=2000, height=1200, margin=2,background_color="white", max_words=100, 
               stopwords=stopword_cloud)
wc.generate(neg_tweets)

In [ ]:
#wc.recolor(color_func = grey_color_func)
import matplotlib.pyplot as plt
plt.figure( figsize=(20,10) )
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

# Step 4: Topic Modelling 

### If we only focus on negative reviews about customer service, we combine data from twitter and facebook:

In [460]:
# get negative customer review from twitter
cs_tt=[]
count=0
for i in neg_review:
    if 'service' in i:
        cs_tt.append(i)
        count+=1
count

163

In [461]:
# get facebook negative posts
neg_fb=[]
with open('neg_cs_fb.txt','r',encoding='utf-8') as f:
    neg_fb=f.readlines()
count=0
for i in neg_fb:
    count+=1
count

5080

In [462]:
# get facebook customer services posts
count=0
cs_fb=[]
for i in neg_fb:
    if 'service' in i:
        cs_fb.append(i)
        count+=1
count

887

In [463]:
# combine them
cs=[]
cs=cs_tt+cs_fb
count=0
for i in cs:
    count+=1
count

1050

In [522]:
for i in cs:
    with open('cs.txt','a',encoding='utf-8') as f:
        f.write(i)
        f.write('\n')

In [500]:
# create stopwords for topic modelling, keep updating according to output 
# (compared to stopwords for frequency analysis, this list adds a lot of adjectives)
stopword_basic=['few', 'want', 'be', 'itself', 'we', 'here', 'so', 'wha', 'their', 'let',
            'yer', 'they', 'have', 'iep', 'didn', 'should', 'yay', 'his', 'I', 'doing',
            'or', 'only', 'an', 'same', 'going', 'yourselves', 'above', 'some', 'thank',
            'will', 'against', 'off', 'having', 'very', 'thanks', 'than', 'and', 'under',
            'more', 'with', 'no', 'get', 'when', 'ours', 'other', 't', 'most', 'my',
            'awesome', 'being', 'once', 'know', 'use', 'if', 'herself', 'like', 'nice', 
            'during', 'don', 'hey', 'down', 'its', 'of', 's', 'there', 'nzdt', 'our', 'was',
            'not', 'too', 'say', 'themselves', 'him', 'could', 'who', 'while', 'just', 'com',
            'ourselves', 'myself', 'yep', 'those', 'top', 'been', 'in', 'that', 'a', 'yours',
            'this', 'wow', 'new', 'the', 'par', 'theirs', 'am', 'amp', 'aest', 'does', 'what',
            'into', 'make', 'is', 'it', 'them', 'such', 'were', 'can', 'why', 'yourself', 'best',
            'after', 'are', 'pln', 'where', 'at', 'pst', 'hoo', 'which', 'got', 'before', 'hers', 
            'nor', 'then', 'own', 'how', 'further', 'hmm', 'had', 'but', 'as', 'he', 'to', 'her',
            'between', 'one', 'me', 'about', 'each', 'whom', 'your', 'by', 'pdt', 'out', 'via',
            'below', 'hop', 'ooo', 'because', 'through', 'yes', 'did', 'you', 'has', 'really', 
            'see', 'until', 'would', 'from', 'again', 'these', 'all', 'himself', 'great', 'i',
            'both', 'now', 'up', 'for', 'good', 'on', 'she', 'over', 'do', 'any','via','late']
stopword_update=['currys','curryspcworld','pc','world','teamknowhowuk','pcworld','teamknowhow','www','http','https',
                'today','american','ve','day','told','buy','pay','company','knowhow','gb','news','mm','store',
                'order','yesterday','money','useless','received','customer','service','bad','poor',
                'shocking','terrible','absolutely','disappointed','appalled','awful','worst','customerservice',
                'customerservices','poorservice','badcustomerservice','badservice','poorcustomerservice','appalling',
                'curry','extremely','speak','said','dreadful','ridiculous','frustrating','disappointing',
                'crap','given','help','non','ever','experienced','paid','even','back','still','team','never','existent',
                'item','way','guy','using','getting','think','come','online','avoid','also','level']
stopword_topic=stopword_basic+stopword_update

In [465]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
wn_lem = WordNetLemmatizer()

In [501]:
# tokenize, lemmatize, remove stopwords and words with one letter
tokens=[]
for review in cs:
    token = nltk.word_tokenize(review)
    tokens.append(token)
lemma_tokens=[]
for review in tokens:
    lemma_review=[]
    for word in review:
        lemma_review.append(wn_lem.lemmatize(word))
    filtered_words = [word for word in lemma_review if word not in stopword_topic and len(word)>2]
    lemma_tokens.append(filtered_words)


In [502]:
# Remove words that only occur once to reduce computation later
from collections import defaultdict
token_frequency = defaultdict(int)

# count all token
for review in lemma_tokens:
    for word in review:
        token_frequency[word] += 1

# keep words that occur more than once
lemma_tokens = [ [word for word in review if token_frequency[word] > 1]
                for review in lemma_tokens  ]


Combine cleaned tokens into tweets, as TfidfVectorizer only deals with string

In [503]:
tweets=[]
for review in lemma_tokens:
    if review!=[]:
        tweet=''
        for word in review:
            tweet=tweet+' '+word
    tweets.append(tweet)

In [ ]:
for i in tweets:
    if 'frustrating' in i:
        print(i)

##### But it seems only gensim can calculate coherence score so we build another lda model called lda_model to decide the optimal topics before building real models

In [469]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [470]:
dictionary = Dictionary(tokens)
dictionary.filter_extremes(no_below=10, no_above=0.2)
corpus = [dictionary.doc2bow(tweet) for tweet in tokens]

In [ ]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:

# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokens, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

3: Coherence Score:  0.32  
4: Coherence Score:  0.38  
5: Coherence Score:  0.38  
6: Coherence Score:  0.39  
7: Coherence Score:  0.36  
8: Coherence Score:  0.38

#### from the result, we will try 4/5/6 topics

##### Start topic modelling

create vector matrix

In [472]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [473]:
# restrict the dimentionality of features
n_features = 1000

In [504]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=stopword_topic,
                                )
tf = tf_vectorizer.fit_transform(tweets)

Train LDA models

In [475]:
from sklearn.decomposition import LatentDirichletAllocation

In [505]:
# decide number of topics
n_topics = 6
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [506]:
# train the data. There will be warnings but just ignore them.
lda.fit(tf)

C:\Users\xueli\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=6, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

Visualize the output.The graph is interactive.

In [478]:
import pyLDAvis
import pyLDAvis.sklearn

In [507]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.030704 -0.081376       1        1  26.329311
5      0.016711 -0.198168       2        1  20.371767
3      0.212488  0.070127       3        1  19.927189
4     -0.096605  0.011511       4        1  16.682330
1     -0.152726  0.041410       5        1   8.446345
2     -0.010572  0.156496       6        1   8.243058, topic_info=    Category        Freq       Term       Total  loglift  logprob
231  Default  130.000000   delivery  130.000000  30.0000  30.0000
171  Default   58.000000  complaint   58.000000  29.0000  29.0000
118  Default   99.000000       call   99.000000  28.0000  28.0000
358  Default   54.000000    freezer   54.000000  27.0000  27.0000
360  Default   71.000000     fridge   71.000000  26.0000  26.0000
524  Default   49.000000    manager   49.000000  25.0000  25.0000
826  Default   58.000000      staff   58.000000  24.0000  24.0000
649  Default   32.000000    product   32.000000  23.0000  23.0000
986  Default   17.000000      worse   17.000000  22.0000  22.0000
157  Default   20.000000       code   20.000000  21.0000  21.0000
271  Default   28.000000     driver   28.000000  20.0000  20.0000
422  Default   51.000000       hour   51.000000  19.0000  19.0000
230  Default   42.000000  delivered   42.000000  18.0000  18.0000
74   Default   26.000000       away   26.000000  17.0000  17.0000
185  Default   18.000000  contacted   18.000000  16.0000  16.0000
765  Default   29.000000       sent   29.000000  15.0000  15.0000
616  Default   96.000000      phone   96.000000  14.0000  14.0000
746  Default   21.000000       rude   21.000000  13.0000  13.0000
953  Default   24.000000    washing   24.000000  12.0000  12.0000
287  Default   36.000000      email   36.000000  11.0000  11.0000
518  Default   31.000000    machine   31.000000  10.0000  10.0000
728  Default   33.000000   response   33.000000   9.0000   9.0000
892  Default   15.000000   tracking   15.000000   8.0000   8.0000
583  Default   22.000000        old   22.000000   7.0000   7.0000
890  Default   14.000000      track   14.000000   6.0000   6.0000
749  Default   28.000000       sale   28.000000   5.0000   5.0000
412  Default   30.000000       hold   30.000000   4.0000   4.0000
483  Default   21.000000     laptop   21.000000   3.0000   3.0000
544  Default   32.000000     minute   32.000000   2.0000   2.0000
571  Default   39.000000     number   39.000000   1.0000   1.0000
..       ...         ...        ...         ...      ...      ...
742   Topic6    1.767755  rochester    2.513600   2.1438  -5.8943
267   Topic6    1.807046     double    2.588131   2.1366  -5.8723
770   Topic6    3.471955     served    5.014087   2.1283  -5.2193
771   Topic6    3.446699        set    4.982254   2.1273  -5.2266
304   Topic6    6.406156   exchange    9.878609   2.0627  -4.6068
987   Topic6    8.401503      worth   13.493664   2.0220  -4.3356
746   Topic6   11.848012       rude   21.263008   1.9110  -3.9919
964   Topic6    5.902616       well    9.967732   1.9718  -4.6886
483   Topic6   11.147844     laptop   21.773453   1.8264  -4.0528
176   Topic6    3.548420  condition    5.949099   1.9791  -5.1975
830   Topic6    4.895401      start    9.137690   1.8717  -4.8757
221   Topic6    2.708397      dealt    4.281623   2.0378  -5.4677
760   Topic6    4.461485       sell    8.335946   1.8707  -4.9685
610   Topic6    3.957752     paying    7.471488   1.8604  -5.0884
551   Topic6    6.015192      month   16.048973   1.5144  -4.6697
765   Topic6    6.752073       sent   29.191568   1.0318  -4.5542
261   Topic6    4.268977      doesn   10.817469   1.5660  -5.0127
524   Topic6    8.038209    manager   49.098672   0.6862  -4.3798
119   Topic6    7.862402     called   46.813299   0.7117  -4.4019
20    Topic6    4.220997    advised   10.691309   1.5664  -5.0240
710   Topic6    5.469370     repair   20.594696   1.1699  -4.7649
450   Topic6    3.397259  i

To build and interpret this model:  
1) decide topic numbers  
2) list the complete reviews in terms of tokens in order to understand the reasons behind (as the code below)

In [521]:
count=0
for tweet in cs:
    if 'track' in tweet:
        count+=1
        print('--------')
        print(tweet)
print('\n','total number:',count)

--------
 dhlparceluk   officialukmail   can your customer service get back to me  why are you issuing curry s customers with tracking codes that are not recognised on your website  this is a serious issue  ukmail  currys poor service  no response or acknowledgement or way to contact you 
--------
waiting for washing machine delivery from  curryspcworld rang to track delivery only to be told the order hasn t been validated so wont be until next week  i am fuming wasted all day waiting in for them  cancelled and gone to  ao for a much better service   currys  fail 
--------
 teamknowhowuk please tell me why my delivery               didn t take place as advised yesterday and is now planned for sunday on track it  i wasted a days leave yesterday and now have to stay in sunday   shockingservice  currys  teamknowhow  fuming 
--------
very bad service  bought a fridge freezer in    agreed to deliver on     received confirmation fo the deliver between   and   took half day of  waited all aft